In [16]:
orcid = '0009-0002-8699-4087' # Fill your orcid here

In [17]:
import requests

We use the `/works` api to list all works related to the orcid. This gives a summary of all works, so citation information is not included. We collect the `put-code` of all works to retrieve the citation information later.

In [18]:
response = requests.get('https://pub.orcid.org/v3.0/{}/works'.format(orcid),
                        headers={"Accept": "application/orcid+json" })
record = response.json()

In [19]:
put_codes = []
for work in record['group']:
    put_code = work['work-summary'][0]['put-code']
    put_codes.append(put_code)
put_code = put_codes[0]

We use the `/<orcid>/work/<put-code>` endpoint to retrieve the citation information for each record.

In [20]:
citations = []
for put_code in put_codes:
    response = requests.get('https://pub.orcid.org/v3.0/{}/work/{}'.format(orcid, put_code),
                            headers={"Accept": "application/orcid+json" })
    work = response.json()
    if work['citation'] is not None:
        citations.append(work['citation']['citation-value'])

In [21]:
# 在写文件前处理重复 key
def dedup_bib_keys(bib_entries):
    """
    检查 BibTeX 字符串列表中的重复 key。
    - 如果某个 key 只出现一次，则保持不变；
    - 如果同 key 出现多次，则第1篇加 a，后续加 b/c/d/...；
    """
    import re
    from collections import defaultdict

    pattern = re.compile(r'(@\w+\s*\{\s*)([^,\s]+)', flags=re.MULTILINE)
    all_keys = []
    key_counts = defaultdict(int)

    # 先收集所有 key 出现次数
    for entry in bib_entries:
        m = pattern.search(entry)
        if m:
            key = m.group(2)
            key_counts[key] += 1
            all_keys.append(key)

    # 再进行改名
    used_indices = defaultdict(int)
    deduped_entries = []

    for entry in bib_entries:
        m = pattern.search(entry)
        if not m:
            deduped_entries.append(entry)
            continue

        prefix, key = m.group(1), m.group(2)
        count = key_counts[key]
        if count == 1:
            # ✅ 仅一篇，不改名
            deduped_entries.append(entry)
        else:
            # ✅ 多篇情况：第1篇 -> a，第2篇 -> b ...
            used_indices[key] += 1
            suffix_index = used_indices[key]
            suffix = chr(96 + suffix_index)  # 1→a, 2→b, ...
            new_key = f"{key}{suffix}"
            print(f"[dedup] {key} (#{suffix_index}) → {new_key}")
            new_entry = pattern.sub(f"{prefix}{new_key}", entry, count=1)
            deduped_entries.append(new_entry)

    return deduped_entries

# 去重处理
citations = dedup_bib_keys(citations)

[dedup] Zhai2024 (#1) → Zhai2024a
[dedup] Zhai2024 (#2) → Zhai2024b
[dedup] Zhai2024 (#3) → Zhai2024c


In [22]:
with open('output.bib', 'w') as bibfile:
    for citation in citations:
        bibfile.write(citation)
        bibfile.write('\n')